In [1]:
import os
import sys
from importlib import reload
from timeit import timeit

## This should then somehow be done internally in heasoftpy, right?  Otherwise 
##  none of the below works for me.  If I don't do this, then I get:
##  
##  ERROR: Device not configured
##  Task ftverify 0.0 terminating with status 6
##  Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)
##  
os.environ["HEADASNOQUERY"]="True"

# General comments

* how to capture output for use in further analysis in python?
* how to work in a notebook? (STDOUT, STDERR issue?)
 * these get returned into the results.STDOUT and results.STDERR attributes
* how to include tools which have no parameter files (i.e. perl tools like ``nicertimeconv``)
* docstrings and other documentation
* parameter handling:
 * take parameter values from the user pfile or system pfile?  
 * allow user to set the pfiles directory?
 * use ``overwrite_pfiles = False`` (or ``batchmode = True``) switch (to keep tasks from updating the user's pfiles) for batch use
 * prompt for inputs if input missing (if no keyword arguments specified)
 * **are there security issues** with passing parameters to the python functions?
 * **note** all parameters are passed as strings, which should minimize (eliminate?) security issues when running through subprocess
* ``import heasoftpY`` results in creation of the heasoft functions in the ``heasoftpy`` directory, which is messy - move to a subdirectory
 * what happens if someone does ``fdump(infile = "test.fits" clobber=True ";\ls -l")`` - would the shell command run?
* how does the speed of the wrapped functions compare to plain ftools?
* prompt for non-optional parameters?
 

# Decisions
We decided to use the write_fns type of function creation.  Now we can simply ``import heasoftpy as h`` and run (for example) ``h.fdump(infile='test.fits')``

In [2]:
import heasoftpy as h

In [4]:
h.quzcif?

# Time Test

In [54]:
infile = os.path.join('/HEA_dev/asdgit/heasoftpy/tmp/testdata/2626010301/auxil','ni2626010301.mkf3')
stmt=f'import write_fns as wtf; \na=wtf.fdump(infile="{infile}", page="no", outfile="STDOUT", rows="10", columns="-")'
print(stmt)


a=wtf.fdump(infile="/HEA_dev/asdgit/heasoftpy/tmp/testdata/2626010301/auxil/ni2626010301.mkf3", page="no", outfile="STDOUT", rows="10", columns="-")
t = timeit(stmt=stmt, number=1)
print(f"This execution took {t:.3f} seconds")
cmdline = '% time fdump 2626010301/auxil/ni2626010301.mkf3 page=no outfile="STDOUT" rows=10 columns="-"'
print(f"\nCommand Line Execution of ")
print(f"{cmdline} was 0.014s")

print(f"\nheasoftpy was a factor of {0.041/0.014:.1f} slower")

import write_fns as wtf; 
a=wtf.fdump(infile="/HEA_dev/asdgit/heasoftpy/tmp/testdata/2626010301/auxil/ni2626010301.mkf3", page="no", outfile="STDOUT", rows="10", columns="-")
This execution took 0.042 seconds

Command Line Execution of 
% time fdump 2626010301/auxil/ni2626010301.mkf3 page=no outfile="STDOUT" rows=10 columns="-" was 0.014s

heasoftpy was a factor of 2.9 slower


# Older Stuff

# ``write_fns`` Package test

Advantages
* straightforward/fast to create functions on import
* allows use the capability of using the defined functions as templates for further analysis

Disadvantages
* will it continue to try to create functions if function creation fails initially?

In [2]:
import write_fns as wtf

In [52]:
a = wtf.fhelp(task="fdump", online="no")
print(a)

In [2]:
a=wtf.fdump(infile="/HEA_dev/asdgit/heasoftpy/tmp/testdata/2626010301/auxil/ni2626010301.mkf3")

NameError: name 'wtf' is not defined

In [29]:
wtf.fdump?

"SIMPLE  =                    T / conforms to FITS standard\nBITPIX  =                    8 / array data type\nNAXIS   =                    0 / number of array dimensions\nEXTEND  =                    T\nEND\n \nXTENSION= 'BINTABLE'           / binary table extension\nBITPIX  =                    8 / array data type\nNAXIS   =                    2 / number of array dimensions\nNAXIS1  =                 3138 / length of dimension 1\nNAXIS2  =                 2317 / length of dimension 2\nPCOUNT  =                    0 / number of group parameters\nGCOUNT  =                    1 / number of groups\nTFIELDS =                   96 / number of table fields\nEXTNAME = 'PREFILTER'          / binary extension contains filter data\nDATE-OBS= '2019-04-18T23:46:00' / Start date of observations\nTIME-OBS= '23:45:59'           / Start time for data\nDATE-END= '2019-04-19T00:24:40' / End date of observations\nTIME-END= '00:24:40'           / End time for data\nTSTART  =           167096762. / As in 

# QUZCIF

In [51]:
wtf.quzcif(mission='rosat')

' ERROR - rcnfig 2.0.1: Unable to find a valid entry for the ROSAT XTI in the Cal\n         db configuration file /software/caldb/caldb.config.20180925\n ERROR - rcnfig 2.0.1: Aborting\n ERROR - caldb_info 1.2.1: CALDB NOT correctly configured for the XTI instrument\n          onboard ROSAT\n ERROR - caldb_info 1.2.1: Serious problem with CALDB set-up\n ERROR - gtcalf 1.2.1: CALDB not defined/available\n ERROR - gtcalf 1.2.1: Fatal - aborting\n'

# Add parameter list to function help

## Function to create a docstring from a parfile

# Test run of nicerl2

In [5]:
wtf.nicerl2?

## Comments

* can't use "clobber+" syntax or positional inputs
* in a Notebook, immediately get `Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)` and `Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)`; ipython command line works ok
* should return print(task_out) rather than task_out to preserve formatting


In [8]:
ddir = 'tmp/testdata'
obsid = '2626010301'
indir = os.path.join(ddir, obsid)
print(wtf.nicerl2(indir=indir, clobber='yes', chatter=5))

['nicerl2', 'indir=tmp/testdata/2626010301', 'clobber=yes']
nicerl2 1.8
--------------------------------------------------------
      ang_dist = 0.015
      br_earth = 30
      calstation = FLIGHT
      cldir = xti/event_cl
      cleanup = 1
      cleanup_ufa_files = 0
      clfile = $CLDIR/ni$OBSID_0mpu7_cl.evt
      clobber = 1
      cor_range = *-*
      elv = 15
      gtifiles = NONE
      history = 1
      indir = tmp/testdata/2626010301
      issmanfile = CALDB
      leapinit = AUTO
      min_fpm = 7
      mkfile = auxil/ni$OBSID.mkf
      mpulist = 0-6
      nicercal_filtexpr = EVENT_FLAGS=bxxxx00
      nicerclean_args = NONE
      nicersaafilt = YES
      nimaketime_gtiexpr = NONE
      niprefilter2 = 1
      overonly_expr = 1.52*COR_SAX**(-0.633)
      overonly_range = *-1.0
      picalfile = CALDB
      pifastcalfile = CALDB
      pirange = 20:1500
      robofile = CALDB
      saafilt = NO
      saaregfile = NONE
      st_valid = YES
      timebiascalfile = CALDB
      track

# Test of fstruct



In [ ]:

file = 'tmp/testdata/2626010301/xti/event_cl/ni2626010301_0mpu7_cl.evt'
a = wtf.fstruct(infile=file)
print(a)


## nicertimeconv

* this is a perl script with no parameter file so there's no par file so no python wrapper is generated
* need to decide how to handle this case - may be generate wrappers for all files in ``$HEADAS/../x86_64-apple-darwin18.6.0/bin`` directory?

In [22]:
wtf.nicertimeconv?

Object `wtf.nicertimeconv` not found.


# Onthefly Testing

* Would it make better sense to move the class definition out of the __init__.py file and into its own package file? 
 * I've done that as a test - moved the class def and functions to ``onthefly.Heaspy`` - I think this makes instantiation more standard
* missing docstrings from the heasoft attributes

In [2]:
from onthefly.Heaspy import Heaspy 

heaspy_dir = /HEA_dev/asdgit/heasoftpy/onthefly


In [3]:
hsp = Heaspy()

## ftverify test

In [8]:
hsp.ftverify?

entering __getattr__ for  ftverify


In [7]:
testfile = !ls tmp/testdata/2626010301/auxil/ni2626010301.mkf3
testfile = testfile[0]
print(hsp.ftverify(infile='testfile'))

Could not find par file for __file__
Could not find par file for __file__
ftverify infile=testfile
 
               ftverify 4.19 (CFITSIO V3.470)               
               ------------------------------               
 
HEASARC conventions are being checked.
 
File: tmp/testdata/2626010301/auxil/ni2626010301.mkf3

2 Header-Data Units in this file.
 
=================== HDU 1: Primary Array ===================
 
 5 header keywords
 
 Null data array; NAXIS = 0 
 
=================== HDU 2: BINARY Table ====================
 
*** Warning: Data checksum is not consistent with  the DATASUM keyword
*** Warning: HDU checksum is not in agreement with CHECKSUM.
 
 409 header keywords
 
 PREFILTER  (96 columns x 2317 rows)
 
 Col# Name (Units)       Format
   1 TIME (s)             1D        
   2 POSITION (km)        3E        
   3 VELOCITY (km/s)      3E        
   4 QUATERNION           4E        
   5 PNTUNIT              3E        
   6 POLAR (rad, rad, km) 3E        
   7 RA (deg)  